In [1]:
import ee
ee.Initialize ()

In [2]:
from iteru import *
from IPython.display import display
import ipywidgets
import datetime

In [3]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [4]:
aoi = Map.aoi

In [5]:
start_year = 2020
start_month = 6
start_day = 1
end_year = 2022
end_month = 2
end_day = 23
time_delta = 20

In [6]:
start_date = datetime.date(start_year,start_month,start_day)
end_date = datetime.date(end_year,end_month,end_day)
ee_start_date = f'{start_year}-{start_month}-{start_day}'
ee_end_date = f'{end_year}-{end_month}-{end_day}'

In [7]:
dates_list = get_dates_list(start_date ,end_date, time_delta)

In [8]:
SAR = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('resolution_meters',10))\
    .filterBounds(aoi)\
    .filterDate(ee_start_date,ee_end_date)\
    .select(['VV','VH'])

In [9]:
SAR = SAR.map(addRatioBand)

In [10]:
Map.add_layer_widgets(SAR,{'min':[-25,-25,0],'max':[0,0,2],'bands':['VV','VH','VV/VH']},'SAR')

SyntaxError: invalid syntax (Temp/ipykernel_10896/2822300032.py, line 1)

In [ ]:
def get_images(day_date):
    
    start_day = ee.Date(day_date)
    end_day = start_day.advance(time_delta,'day')
    
    return SAR.filterDate(start_day,end_day).reduce(ee.Reducer.median())\
          .set({'start_day':start_day.format('YYYY-MM-dd')})\
          .set({'end_day':end_day.format('YYYY-MM-dd')})

In [ ]:
images = ee.List(dates_list).map(get_images)

In [ ]:
collection = ee.ImageCollection.fromImages(images)

In [ ]:
videoArgs = {
  'dimensions': 900 ,
  'region': aoi,
  'framesPerSecond':1,
  'crs': 'EPSG:3857',
  'min':-25,
  'max':5,
  'bands':['VV_median','VH_median','VH_median']
  
        }

In [ ]:
url = collection.getVideoThumbURL(videoArgs)

In [ ]:
out_gif = get_gif(url)

In [ ]:
add_text_to_gif(out_gif,dates_list, dates_font_size = 25, dates_font_color = 'black', framesPerSecond = 1 )

In [ ]:
display_gif(out_gif)

In [ ]:
#x = [string_to_datetime(date) for date in images_dates [62:] ]

In [ ]:
# empty_date = '2020-12-14'

In [ ]:
# def string_to_datetime(string_date):
#     year = int(string_date.split('-')[0])
#     month = int(string_date.split('-')[1])
#     day = int(string_date.split('-')[2])
    
#     d_date = datetime.date(year,month,day)
    
#     return d_date
    
    

In [ ]:
# def nearest(items, pivot):
#     return min(items, key=lambda x: abs(x - pivot))

In [ ]:
#nearest(x,string_to_datetime(empty_date))